<a href="https://colab.research.google.com/github/SatpreetMakhija/Titanic-Kaggle/blob/main/Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Semesters/Data Science/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Semesters/Data\ Science/Kaggle

/content/drive/MyDrive/Semesters/Data Science/Kaggle


In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 424kB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 13.7MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 3.89MB/s]


## Import Libraries

Since, this is my first Kaggle notebook, I will briefly explain the usage of libraries imported for future reference. 

In [ ]:
import sys #gives access to system parameters
import pandas as pd #data processing and analysis modeled after R dataframes with SQL like features
import matplotlib #scientific visualization
import numpy as np #used for scientific computing
import scipy as sp #scientific computing and advanced maths
import IPython
from IPython import display #pretty printing of dataframes 
import sklearn #collection of ML algorithms
import time
import random

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
print('Python version: {}'.format(sys.version))

-------------------------
Python version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]


## Say Hello to Data

In [ ]:
#import data
data_raw = pd.read_csv('train.csv')
data_val = pd.read_csv('test.csv')

#we will explore the data, so we create a copy
data1 = data_raw.copy(deep = True)

#we do the following for convenient cleaning of both datasets, raw and it's copy at once
data_cleaner = [data1, data_val]

#preview data
print(data_raw.info())
data_raw.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
687,688,0,3,"Dakic, Mr. Branko",male,19.0,0,0,349228,10.1708,NaN,S
491,492,0,3,"Windelov, Mr. Einar",male,21.0,0,0,SOTON/OQ 3101317,7.2500,NaN,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.2500,NaN,S
327,328,1,2,"Ball, Mrs. (Ada E Hall)",female,36.0,0,0,28551,13.0000,D,S
308,309,0,2,"Abelson, Mr. Samuel",male,30.0,1,0,P/PP 3381,24.0000,NaN,C
177,178,0,1,"Isham, Miss. Ann Elizabeth",female,50.0,0,0,PC 17595,28.7125,C49,C
181,182,0,2,"Pernot, Mr. Rene",male,NaN,0,0,SC/PARIS 2131,15.0500,NaN,C
451,452,0,3,"Hagland, Mr. Ingvald Olai Olsen",male,NaN,1,0,65303,19.9667,NaN,S
207,208,1,3,"Albimona, Mr. Nassef Cassem",male,26.0,0,0,2699,18.7875,NaN,C
353,354,0,3,"Arnold-Franchi, Mr. Josef",male,25.0,1,0,349237,17.8000,NaN,S


### Data Description
1. _Survived_ : Binary; 1 for survived and 0 for did not survive
2. _PassengerID_ & _Ticket_: Random unique identifiers; can be ignored as they do not influence the 'survivability'. 
3. _Pclass_: describes socio-economic status(SES); 1 = upper class, 2 = middle class, 3 = lower class
3. _Name_: It can be used in feature engineering to derive gender from title, family size from surname and SES from titles like doctor or master.
4. _Sex_ & _Embarked_: nominal datatype; will be converted to dummy variables for mathematical calculations. The latter signifies whether the person boarded the ship or not (I think). 
5. _Age_ & _Fare_: self-explanatory.
6. _SibSp_: #related siblings/spouse aboard
7. _Parch_: #related parents/children aboard
8. _Cabin_: nominal value; although can be used in feature engineering for approximate position on ship but most entries are Null values; hence, excluded from analysis

## Data Cleaning: Correcting, Completing, Creating, Coverting
1. __Correcting__: check for non-plausible values. Eg: age is 800.
2. __Completing__: check for null values. Two approaches to solve the problem. First, delete the entry. Second, impute missing values. Use mean, median, mode to do so. 
3. __Creating__: Feature engineering is when we use existing features to create new features to determine if they provide new signals to predict our outcome. Here, we will create _title_ feature to determine if they played a role in survival. 
4. __Converting__: Deal with formatting data. 

In [ ]:
print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')

Train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------
Test/Validation columns with null values:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
----------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Stranden, Mr. Juho",male,NaN,NaN,NaN,1601,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [ ]:
##COMPLETING: complete or delete missing values in train and test/validation dataset
for dataset in data_cleaner:
  #complete missing age with median
  dataset['Age'].fillna(dataset['Age'].median(), inplace = True)

  #complete embarked with mode
  dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

  #complete missing fare with median
  dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)


#delete some not so useful features/columns
drop_column = ['PassengerId', 'Cabin', 'Ticket']
data1.drop(drop_column, axis = 1, inplace = True)

print(data1.isnull().sum())
print('-'*10)
print(data_val.isnull().sum())

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [ ]:
data1

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,23.4500,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C


In [ ]:
#CREATE: Feature Engineering for train and test/validation dataset
for dataset in data_cleaner:
  #Discrete variables
  dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
  dataset['IsAlone'] = 1 #initialize to yes/1 is alone
  dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 #updates to no/0 if family size is > 1
  #quick code to split title from 'Name'
  dataset['Title'] = dataset['Name'].str.split(", ", expand = True)[1].str.split(".", expand = True)[0]

  #hello
  

